In [59]:
import pandas as pd
import numpy as np
SHIFT_WEEK = 2

In [12]:
filename = "2230.csv"

df = pd.read_csv(filename, index_col=False)
df

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,100/01/03,"35,747,205","2,541,551,155",71.5,71.6,70.8,71.1,+0.10,"7,394"
1,100/01/04,"36,048,199","2,567,399,929",71.0,71.6,70.8,71.2,+0.10,"8,813"
2,100/01/05,"54,327,088","3,811,818,734",70.9,71.1,69.6,69.8,-1.40,"12,673"
3,100/01/06,"46,802,332","3,295,606,915",69.8,71.0,69.7,71.0,+1.20,"8,619"
4,100/01/07,"170,328,819","12,335,418,871",71.9,73.0,71.9,72.5,+1.50,"23,691"
...,...,...,...,...,...,...,...,...,...
3233,113/06/24,"74,081,708","70,162,177,693",958.0,960.0,940.0,940.0,-30.00,"227,255"
3234,113/06/25,"62,079,701","57,978,440,842",925.0,945.0,923.0,945.0,+5.00,"103,844"
3235,113/06/26,"48,821,135","46,668,874,056",951.0,960.0,951.0,960.0,+15.00,"56,563"
3236,113/06/27,"41,276,086","39,447,006,518",951.0,961.0,949.0,960.0,0.00,"41,133"


In [ ]:
convert_cols = ["成交股數",	"成交金額",	"開盤價", "最高價",	"最低價", "收盤價", "漲跌價差", "成交筆數"]
df["漲跌價差"] = df["漲跌價差"].replace("X", '', regex=True)
df[convert_cols] = df[convert_cols].replace(',', '', regex=True).astype(float)
df["日期"] = df["日期"].apply(lambda x: f"{int(x.split('/')[0]) + 1911}/{x.split('/')[1]}/{x.split('/')[2]}")
df["日期"] = pd.to_datetime(df["日期"], format="%Y/%m/%d")


In [19]:
df

,日期,成交股數,成交金額,開盤價,最高價,最低價,收盤價,漲跌價差,成交筆數
0,2011-01-03,35747205.0,2.541551e+09,71.5,71.6,70.8,71.1,0.1,7394.0
1,2011-01-04,36048199.0,2.567400e+09,71.0,71.6,70.8,71.2,0.1,8813.0
2,2011-01-05,54327088.0,3.811819e+09,70.9,71.1,69.6,69.8,-1.4,12673.0
3,2011-01-06,46802332.0,3.295607e+09,69.8,71.0,69.7,71.0,1.2,8619.0
4,2011-01-07,170328819.0,1.233542e+10,71.9,73.0,71.9,72.5,1.5,23691.0
...,...,...,...,...,...,...,...,...,...
3233,2024-06-24,74081708.0,7.016218e+10,958.0,960.0,940.0,940.0,-30.0,227255.0
3234,2024-06-25,62079701.0,5.797844e+10,925.0,945.0,923.0,945.0,5.0,103844.0
3235,2024-06-26,48821135.0,4.666887e+10,951.0,960.0,951.0,960.0,15.0,56563.0
3236,2024-06-27,41276086.0,3.944701e+10,951.0,961.0,949.0,960.0,0.0,41133.0


In [62]:
def five_day_cut(df: pd.DataFrame, using_cols: list)-> list:
    """
    每連續五天的工作日放在同一塊
    """

    days = list(df["日期"].dt.day)
    five_day_data = []
    tmp = 0
    last_day = days[0]
    
    for i in range(1, len(days)):
        if i - tmp == 5:
            five_day_data.append(list(np.array(df.iloc[tmp:i][using_cols])))
        if days[i] - last_day != 1: #不存在跨月同個禮拜的情況
            tmp = i
        last_day = days[i]
    return five_day_data







In [70]:
using_cols = ["開盤價", "最高價", "最低價", "收盤價", "漲跌價差"]

five_day_data = five_day_cut(df, using_cols)


In [92]:
def get_data_pair(five_day_data, shift_week=1):
    _data = five_day_data[0: len(five_day_data) - shift_week]
    _label = five_day_data[shift_week: len(five_day_data)]
    data = []
    label = []

    for i in range(len(_data) - shift_week):
        new_data = _data[i].copy()
        new_label = _label[i].copy()

        for j in range(i + 1, i + shift_week):
            new_data.extend(_data[j].copy())
            new_label.extend(_label[j].copy())
        data.append(np.array(new_data))
        label.append(np.array(new_label)[:, 3])
    return data, label

data, label = get_data_pair(five_day_data, 2)

print(data[2], label[0])

[[75.4 75.9 75.3 75.8  1. ]
 [75.5 77.4 75.2 76.8  1. ]
 [76.9 78.3 76.6 78.   1.2]
 [77.  77.  76.1 76.4 -1.6]
 [74.  75.2 74.  74.6 -1.8]
 [72.  72.9 71.5 72.5  1.1]
 [72.4 73.1 71.4 72.5  0. ]
 [72.4 72.7 71.9 72.1 -0.4]
 [72.4 72.9 71.6 72.5  0.4]
 [73.1 73.9 73.  73.   0.5]] [75.8 76.8 78.  76.4 74.6 72.5 72.5 72.1 72.5 73. ]


In [86]:
import torch
from torch import nn    #可以呼叫模型、損失函數....
from torch import optim #引入優化器具
import torch.nn.functional as F #可以呼叫模型、損失函數....
from torch.utils.data import TensorDataset, DataLoader, RandomSampler

In [ ]:
data_tensor, label_tensor = torch.tensor(data), torch.tensor(label)
data_label_dataset = TensorDataset(data_tensor, label_tensor)

batch = 64

indice = torch.randperm(len(data_label_dataset))

train_sampler = RandomSampler(indice[:int(0.9 * len(indice))])
test_sampler = RandomSampler(indice[int(0.9 * len(indice)):])

train_dataloader = DataLoader(data_label_dataset, batch_size=batch, sampler=train_sampler)
test_dataloader = DataLoader(data_label_dataset, batch_size=batch, sampler=test_sampler)

In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, hidden_dim=128, tagset_size=10, input_dim=4):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        # LSTM層
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)

        # 線性層
        self.linear = nn.Linear(hidden_dim, tagset_size)
        self.relu = nn.ReLU()

    def init_hidden(self, batch_size):
        # 初始化隱藏狀態和細胞狀態
        return (torch.zeros(1, batch_size, self.hidden_dim),
                torch.zeros(1, batch_size, self.hidden_dim))

    def forward(self, sentence, hidden):
        """
        sentence: Tensor, shape (batch_size, seq_len, input_dim)
        hidden: Initial hidden state

        Returns:
            tag_space: Tensor, shape (batch_size, tagset_size)
            hidden: Final hidden state
        """
        # LSTM層
        lstm_out, hidden = self.lstm(sentence, hidden)

        # 取最後一個時間步的輸出
        last_output = lstm_out[:, -1, :]  # shape: (batch_size, hidden_dim)

        # 線性層和激活函數
        tag_space = self.relu(self.linear(self.relu(last_output)))

        return tag_space, hidden
    

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 定義訓練函數
def train_model(model, train_loader, valid_loader, train_length, valid_length, num_epochs=10, learning_rate=0.001):
    # 將模型移到 GPU
    model = model.to(device, dtype=torch.float32)
    # 使用 Adam 優化器
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    # 定義損失函數
    criterion = nn.SmoothL1Loss()
    min_loss = 1e9

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0.0

        for inputs, labels in train_loader:
            # 將輸入和標籤移到 GPU
            inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32)
            
            # 初始化隱藏狀態
            hidden = model.init_hidden(batch_size=inputs.size(0))
            hidden = tuple([h.to(device, dtype=torch.float32) for h in hidden])

            # 清零梯度
            optimizer.zero_grad()
            # 前向傳播
            outputs, _ = model(inputs, hidden)

            # 計算損失
            loss = criterion(outputs.squeeze(), labels)
            # 反向傳播
            loss.backward()
            # 更新參數
            optimizer.step()

            total_loss += loss.item()

        print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {total_loss / train_length:.4f}")

        # 驗證模型
        total_loss = validate_model(model, valid_loader, criterion, valid_length)
        if total_loss < min_loss:
            min_loss = total_loss
            torch.save(model.state_dict(), "checkpoint.pth" )


# 定義驗證函數
def validate_model(model, valid_loader, criterion, valid_length):
    model.eval()
    total_loss = 0.0
    error = 0
    count = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            # 將輸入和標籤移到 GPU
            inputs, labels = inputs.to(device, dtype=torch.float32), labels.to(device, dtype=torch.float32)
            
            # 初始化隱藏狀態
            hidden = model.init_hidden(batch_size=inputs.size(0))
            hidden = tuple([h.to(device, dtype=torch.float32) for h in hidden])

            # 前向傳播
            outputs, _ = model(inputs, hidden)
            # 計算損失
            loss = criterion(outputs.squeeze(), labels.squeeze())
            error += abs(outputs.view(-1) - labels.view(-1)).sum()
            count += labels.shape[0] * labels.shape[1]
            total_loss += loss.item()

    print(f"Validation Loss: {total_loss / valid_length:.4f}, valid error: {error / count}, count: {count}")
    return total_loss